In [39]:
%matplotlib inline

In [9]:
import sklearn
import nose
import astroML_addons
import astroML
import scipy
import pandas
import xdgmm
from astropy.io import fits
import csv
import os
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt

#from demo_plots import *

import requests
import ast

import warnings
warnings.filterwarnings('ignore')

def fetchFirstVals():
    
    deltaW = []
    sigW = []
    w1 = []
    w2 = []
    w1s = []
    w2s = []
    mjd = []
    name = []
    

    directorystr = './csvfiles/galaxy_csvfiles-150318/'
    directory = os.fsencode(directorystr)
    filelist = os.listdir(directory)
    print(len(filelist))
    readdata = np.zeros((22,len(filelist))) #one big list to rule them all
    fileCount = 0 #to keep our place in each list as we go through the files
    blankCount = 0
    for file in filelist:
        filestr = os.fsdecode(file)
        if(filestr.endswith(".csv")):
            readfile = open('./csvfiles/galaxy_csvfiles-150318/'+filestr, 'r')
            #name.append(filestr)
            reader = csv.reader(readfile)
            typeIndex = 0 #to iterate through the w1, w2, etc list
            nullCount = 0
            rowCount = 0
            try:
                for row in enumerate(reader):
                    entry = row[1][0]
                    try:
                        fentry = float(entry)
                    except ValueError:
                        if entry != 'null':
                            #print("String!")
                            rowCount = rowCount + 1 #keep track of where we are in the file
                            continue #skip to next row, do not increment readdata row
                        else:
                            #print("Null!")
                            nullCount = nullCount + 1 #this is a null in W1 or W2. don't want this file
                    
                    #print(fentry)
                    readdata[typeIndex][fileCount] = fentry
                    rowCount = rowCount + 1 #move on to the next row in the file
                    typeIndex = typeIndex + 1 #move to next row in the data array
                    if rowCount == 14:
                        break #move on to next file.
                if nullCount == 0:
                    fileCount = fileCount + 1 #overwrite if we found null in W1, W2.
                    name.append(filestr)
            except IndexError:
                #print(readdata[0][fileCount])
                #print(filestr)
                #print("IndexError")
                blankCount = blankCount + 1
                continue
                #break
                
    print(blankCount)       
    w1 = [float(x) for x in readdata[1]]
    #print(w1)
    w2 = [float(x) for x in readdata[3]]
    #print(w2)
    w1s = [float(x) for x in readdata[2]]
    #print(w1s)
    w2s = [float(x) for x in readdata[4]]
    #print(w2s)
    
    #print(name)
    
    deltaW = [x - y for x,y in zip(w1,w2)]
    sigW = [np.sqrt(sx**2 + sy**2) for sx,sy in zip(w1s, w2s)]
    
    notNullCount = 0
    for val in w1:
        if val != 0:
            notNullCount = notNullCount +1
    deltaW = deltaW[:notNullCount]
    sigW = sigW[:notNullCount]
    
    return deltaW, sigW, notNullCount

print("starting")
result = fetchFirstVals()
deltaW = result[0]
sigW = result[1]
npoints = result[2]
#print(deltaW)
xdgmm = xdgmm.XDGMM()

param_range = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
deltaW = np.array(deltaW)

deltaW = deltaW.reshape(deltaW.shape[0], 1)
#print(deltaW.shape)
sigW = np.array(sigW)

sigW = sigW.reshape(sigW.shape[0], 1,1)
#print(sigW.shape)

#bic, optimal_n_comp, lowest_bic = xdgmm.bic_test(deltaW, sigW, param_range)

print('yes')


starting
47522
3015
yes


In [ ]:
print(deltaW)

In [5]:
import datetime
#print(datetime.datetime.now().strftime("%Y-%m-%d"))
datestr = datetime.datetime.now().strftime("%Y-%m-%d")

print("starting")
xdgmm.n_components = 9
xdgmm = xdgmm.fit(deltaW, sigW)
print('done')

xdgmm.save_model('galaxyTrialSet'+datestr+'.fit')

starting
done


In [8]:
%matplotlib inline


import sklearn
import nose
import astroML_addons
import astroML
import scipy
import pandas
import xdgmm
from astropy.io import fits
import csv
import os
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt

#from demo_plots import *

import requests
import ast

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(threshold=np.nan)

xdgmm = xdgmm.XDGMM(filename='galaxyTrialSet2018-03-22.fit')         
    
def makeGaussian(x, mu, sig):
    return (np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))/(2.0*np.pi*sig)

binwidth = 0.01

sample = xdgmm.sample(npoints)
plt.figure()
h1 = plt.hist(sample, bins=np.arange(-1, 3, binwidth))
plt.title('XDGMM-resampled W1-W2 band magnitude difference')
plt.xlabel('Magnitude difference')
plt.ylabel('Occurrence')



plt.figure()

#for mu, alpha in xdgmm.mu, xdgmm.weights:
x = np.arange(-0.5, 2.5, 0.01)
plt.plot(x, makeGaussian(x, xdgmm.mu[0], xdgmm.weights[0]))
plt.plot(x, makeGaussian(x, xdgmm.mu[1], xdgmm.weights[1]))
plt.plot(x, makeGaussian(x, xdgmm.mu[2], xdgmm.weights[2]))
plt.plot(x, makeGaussian(x, xdgmm.mu[3], xdgmm.weights[3]))
plt.plot(x, makeGaussian(x, xdgmm.mu[4], xdgmm.weights[4]))
plt.plot(x, makeGaussian(x, xdgmm.mu[5], xdgmm.weights[4]))
plt.plot(x, makeGaussian(x, xdgmm.mu[6], xdgmm.weights[4]))
plt.plot(x, makeGaussian(x, xdgmm.mu[7], xdgmm.weights[4]))
plt.plot(x, makeGaussian(x, xdgmm.mu[8], xdgmm.weights[4]))
plt.title('Gaussian model components')
plt.xlabel('Magnitude difference')
plt.ylabel('Weight')

#print(deltaW)

plt.figure()

plt.hist(deltaW, bins=np.arange(-1, 3, binwidth))
plt.title('True W1-W2 band magnitude difference')
plt.xlabel('Magnitude difference')
plt.ylabel('Occurrence')


NameError: name 'npoints' is not defined

In [82]:

tryProp = xdgmm.score_samples(1.5, 0.03)

TypeError: 'float' object is not subscriptable